# Image OCR

In [0]:
!sudo apt install tesseract-ocr && sudo apt install libtesseract-dev && sudo pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 10 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,825 kB/s)
debconf: unable to initi

In [0]:
!tesseract --version

tesseract 4.0.0-beta.1
 leptonica-1.75.3
  libgif 5.1.4 : libjpeg 8d (libjpeg-turbo 1.5.2) : libpng 1.6.34 : libtiff 4.0.9 : zlib 1.2.11 : libwebp 0.6.1 : libopenjp2 2.3.0

 Found AVX2
 Found AVX
 Found SSE


In [0]:
!tesseract panna.png stdout -l eng --oem 1 --psm 3

Warning. Invalid resolution 0 dpi. Using 70 instead.
Estimating resolution as 541
DON'T
STOP

UNTIL
YOU'RE
PROUD

 


# Summerizer

In [25]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') 
import re

df = pd.read_csv("data.csv", index_col=0)    #common
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,message
id,
1,"Hey,\n\nWhatsApp Messenger is a fast, simple a..."
2,Senior Analytics Manager - Aasaanjobs.pdf\nYou...
3,Analytics Manager - Aasaanjobs.pdf
4,Business Analyst - Campus.pdf
5,Hi guys\nThere are vacancies for Business anal...


In [0]:
from nltk.tokenize import sent_tokenize        #common
sentences = [sent_tokenize(x) for x in df['message']]
sentences = [y for x in sentences for y in x] # flatten list

In [6]:
!wget http://nlp.stanford.edu/data/glove.6B.zip  #once
!unzip glove*.zip

--2019-03-26 11:12:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-03-26 11:12:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  15.3MB/s    in 58s     

2019-03-26 11:13:23 (14.1 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
# Extract word vectors #common
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [10]:
nltk.download('stopwords') #downloading the stop_words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# function to remove stopwords #once
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences  #common
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
import networkx as nx   # Converting sim_mat into a graph

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [35]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Extract top 3 sentences as the summary
for i in range(3):
  print(ranked_sentences[i][1])

Everything is totally free....

After completion of this Training studants can get direct entry to Diploma Engineering in eligible stream

:pray:my humble request to you, if you find good students with financial poor family in our society ( Eg.Our maid's childrens) please communicate below advertisement to them or give below mentioned cell no.
Senior Analytics Manager - Aasaanjobs.pdf
You can even refer your friends for this role.
We are searching such students who are interested in Technical career and due to family’s financial problems they couldn't continue their career.
